In [ ]:
!pip install transformers datasets openpyxl

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers datasets
import pandas as pd
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

def preprocess_function(texts):
    """
    Preprocesses a list of texts by tokenizing them.

    Args:
        texts: A list of texts to preprocess.

    Returns:
        A dictionary containing the tokenized input IDs, attention mask, and token type IDs.
    """
    # Convert any non-string elements to strings
    texts = [str(text) if not isinstance(text, str) else text for text in texts]

    # Filter out None values and then tokenize
    valid_texts = [text for text in texts if text is not None]
    return tokenizer(valid_texts, padding=True, truncation=True, return_tensors='tf')

# Load the data
df = pd.read_excel('/content/drive/MyDrive/train.xlsx')

# Extract features and labels
X = df['comment_text'].tolist()
y = df[df.columns[2:]].values

# Load pre-trained tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = TFAutoModelForSequenceClassification.from_pretrained('sentence-transformers/all-MiniLM-L6-v2', num_labels=y.shape[1])

# Encode the input texts
encoded_X = preprocess_function(X)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
tf_dataset = tf.data.Dataset.from_tensor_slices((dict(encoded_X), y))

In [ ]:
train_size = 0.7
val_size = 0.2
test_size = 0.1

train_dataset = tf_dataset.take(int(len(X) * train_size))
val_dataset = tf_dataset.skip(int(len(X) * train_size)).take(int(len(X) * val_size))
test_dataset = tf_dataset.skip(int(len(X) * (train_size + val_size)))

# Batch and prefetch the datasets
train_dataset = train_dataset.batch(16).shuffle(1000).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
val_dataset = val_dataset.batch(16).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.batch(16).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),  # Using binary crossentropy
    metrics=['accuracy']
)

In [ ]:
history = model.fit(train_dataset, epochs=10, validation_data=val_dataset)


Epoch 1/10
447/447 [==============================] - 275s 487ms/step - loss: 0.1520 - accuracy: 0.8380 - val_loss: 0.0656 - val_accuracy: 0.9956
Epoch 2/10
447/447 [==============================] - 217s 485ms/step - loss: 0.0559 - accuracy: 0.9936 - val_loss: 0.0536 - val_accuracy: 0.9956
Epoch 3/10
447/447 [==============================] - 217s 485ms/step - loss: 0.0436 - accuracy: 0.9934 - val_loss: 0.0574 - val_accuracy: 0.9956
Epoch 4/10
447/447 [==============================] - 217s 486ms/step - loss: 0.0384 - accuracy: 0.9934 - val_loss: 0.0590 - val_accuracy: 0.9956
Epoch 5/10
447/447 [==============================] - 217s 486ms/step - loss: 0.0343 - accuracy: 0.9936 - val_loss: 0.0643 - val_accuracy: 0.9956
Epoch 6/10
447/447 [==============================] - 217s 485ms/step - loss: 0.0327 - accuracy: 0.9880 - val_loss: 0.0605 - val_accuracy: 0.9956
Epoch 7/10
447/447 [==============================] - 216s 484ms/step - loss: 0.0303 - accuracy: 0.9835 - val_loss: 0.0597 -